In [1]:
base_path = '/home/datasets/rsna-intracranial-hemorrhage-detection/'
csv_path = base_path + 'stage_2_train.csv'
items = base_path + 'stage_2_train/'
from torch.utils.data import Dataset, DataLoader
import torch, csv, pydicom
import torchvision.transforms as transforms
from PIL import Image

class RSNASet(Dataset):
    def __init__(self, csv_path, root_dir):
        self.root_dir = root_dir
        labels = {}
        with open(csv_path) as f:
            rdr = csv.reader(f)
            next(rdr)
            for rows in rdr:
                fn = '_'.join(rows[0].split('_')[0:2])
                try:
                    labels[fn].append(int(rows[1]))
                except:
                    labels[fn] = [int(rows[1])]
        for fn in labels.keys():
            # "any" should really be the classification "none"
            # so, it should only be 1 if nothing else is true - swap the values
            labels[fn][5] = abs(labels[fn][5]-1)
        self.labels = labels
        self.files = list(labels.keys())
    def __len__(self):
        return len(self.labels)
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        file = self.files[idx]
        label = torch.argmax(torch.FloatTensor(self.labels[file]))
        img = pydicom.dcmread(self.root_dir+file+'.dcm').pixel_array
        tf = transforms.Compose([
            transforms.ToTensor(),
            transforms.Resize((256, 256)),
            transforms.Lambda(lambda x: x.repeat(3,1,1)),
            transforms.ConvertImageDtype(torch.float),
        ])
        img = tf(Image.fromarray(img))
        return img, label
tempset = RSNASet(csv_path=csv_path, root_dir=items)
BATCH_SIZE = 128
import math
trainlen = math.floor(0.80 * len(tempset)) # 80/20 train/val split
vallen = len(tempset) - trainlen
_, valset = torch.utils.data.random_split(tempset, [trainlen, vallen], generator=torch.Generator().manual_seed(42))
model = 'resnet18'
valloader = torch.utils.data.DataLoader(valset,batch_size=BATCH_SIZE,shuffle=True,num_workers=4)
classes = ('epidural', 'intraparenchymal', 'intraventricular', 'subarachnoid', 'subdural','none')

/home/tools/anaconda3/lib/python3.9/site-packages/torch/utils/data/dataloader.py:478: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [2]:
import glob, os
from torch.utils.data import Dataset, DataLoader
import torch, csv, pydicom
import torchvision.transforms as transforms
from PIL import Image
class RSNATestSet(Dataset):
    def __init__(self, files, root_dir):
        self.root_dir = root_dir
        self.files = files
    def __len__(self):
        return len(self.files)
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        file = self.files[idx]
        img = pydicom.dcmread(file).pixel_array
        tf = transforms.Compose([
            transforms.ToTensor(),
            transforms.Resize((256, 256)),
            transforms.Lambda(lambda x: x.repeat(3,1,1)),
            transforms.ConvertImageDtype(torch.float),
        ])
        img = tf(Image.fromarray(img))
        return idx, img

In [3]:
import torch
from tqdm import tqdm
import torch.nn.functional as F
BATCH_SIZE = 128
criterion = torch.nn.CrossEntropyLoss()

base_path = '/home/datasets/rsna-intracranial-hemorrhage-detection/'
items = base_path + 'stage_2_test/'
dcms = glob.glob('{}*.dcm'.format(items))
testset = RSNATestSet(files=dcms, root_dir=items)
testloader = torch.utils.data.DataLoader(testset,batch_size=BATCH_SIZE,shuffle=True,num_workers=4)
classes = ('epidural', 'intraparenchymal', 'intraventricular', 'subarachnoid', 'subdural', 'none')
model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=False, num_classes=len(classes))
if not torch.cuda.is_available():
    model.load_state_dict(torch.load('weights/split_resnet18/epoch25.pth', map_location=torch.device('cpu')))
else:
    model.load_state_dict(torch.load('weights/split_resnet18/epoch25.pth'))
model.eval()
if torch.cuda.is_available():
    print("CUDA is available - using CUDA!")
    model = model.cuda()
running_loss = 0.0
batch = 0
total_correct = 0
total = 0
with tqdm(valloader, unit="batch") as tepoch:
        model.eval()
        for inputs, labels in tepoch:
            tepoch.set_description(f"Validating")
            if torch.cuda.is_available():
                inputs = inputs.cuda()
                labels = labels.cuda()
            # forward + backward + optimize
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            outputs = F.softmax(outputs, dim=1)
            predictions = outputs.argmax(dim=1, keepdim=True).squeeze()
            correct = (predictions == labels).sum().item()
            total_correct += correct
            total += BATCH_SIZE
            accuracy = total_correct / total # batch_size
            running_loss += loss.item()
            if batch > 0:
                tepoch.set_postfix(loss=running_loss/batch, accuracy=100. * accuracy)
            batch += 1

Using cache found in /home/dsingh/.cache/torch/hub/pytorch_vision_v0.10.0


CUDA is available - using CUDA!


Validating: 100%|██████████| 1177/1177 [22:07<00:00,  1.13s/batch, accuracy=89.2, loss=0.638]
